In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
import numpy as np
import scipy as sp
import pandas as pd
import intake
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
hv.notebook_extension('bokeh')

In [ ]:
import requests
def check_url(path):
    r = requests.head(path)
    return r.status_code == requests.codes.ok



---
## Let's download some examples files

<img src="../assets/data_url.jpg" alt="mint-data-fs" width="500"/>

* The acronyms can be searched on NASA'S Open Data [Catalog](https://data.nasa.gov/browse)
* NASA's EARTHDATA GES DISC hosts this data [here](https://is.gd/a6wP0x)

* Summary:
    - FLDAS: FLDAS(Famine Early Warning Systems Network - FEWS NET) Land Data Assimilation System
    - NOAH, VIC: simulation models 
        - '01', '025': model versions
    - A: Anomaly, C: Climatoloty: what the models are about
    - EA: (aoi) Eastern Africa
    - D or M: (time period) 'Daily' or 'Monthly'

---
---
# HALT!
Modified: May 27, 2019
- Submitted an [issue](https://is.gd/1Iytqf)

In [ ]:
# OpenDAP authroization
from netrc import netrc
username, _, pw = netrc().hosts['urs.earthdata.nasa.gov']

#OPENDAP client
from pydap.client import open_url
from pydap.cas.urs import setup_session

In [ ]:
FLDAS_BASE_URL = 'https://hydro1.gesdisc.eosdis.nasa.gov/opendap/hyrax/FLDAS/'
# resource = 'FLDAS_NOAH01_C_SA_MA.001/2013/FLDAS_NOAH01_C_SA_MA.ANOM201301.001.nc'
models = ['NOAH01', 'VIC025']
aois = ['EA','SA','WA']    

year = 2019
month = 4
day = 1
model = models[0]
aoi = aois[0]
resource = f'FLDAS_{model}_A_{aoi}_D.001/{year}/{month:02}/FLDAS_{model}_A_{aoi}_D.A{year}{month:02}{day:02}.001.nc'
print(resource)

resource_spec = {
    'model':model,
    'aoi': aoi,
    'year':year,
    'month':month,
    'day':day
}
pprint(resource_spec)

In [ ]:
def build_resource_spec(model, aoi, year, month, day):
    """
    - model (str): one of ['NOAH01', 'VIC025']
    - aoi (str): one of ['EA','SA','WA'] 
    - year (int): 4 digit year 
    - month (int): 1 or 2 digits month
    - day (int): 1 or 2 digits day
    """
    return {
        'model':model,
        'aoi': aoi,
        'year':year,
        'month':month,
        'day':day
        }
def test_build_resource_spec():
    year = 2019
    month = 4
    day = 1
    model = models[0]
    aoi = aois[0]
    print(build_resource_spec(model, aoi, year, month, day))
test_build_resource_spec()
          

In [ ]:
# resource = f'FLDAS_{model}_A_{aoi}_D.001/{year}/{month:02}/FLDAS_{model}_A_{aoi}_D.A{year}{month:02}{day:02}.001.nc'
# print(resource)
    
# def build_resource_path(model, aoi, year, month, day):
def build_resource_path(**spec):
    """
    Args:
    - spec (dict) which must contain all of the following as key:value
        - model (str): one of ['NOAH01', 'VIC025']
        - aoi (str): one of ['EA','SA','WA'] 
        - year (int): 4 digit year 
        - month (int): 1 or 2 digits month
        - day (int): 1 or 2 digits day
    Returns:
    - resource (str): FLDAS Daily (Abnormality?) Dataset resource path
    """
    path = f'FLDAS_{model}_A_{aoi}_D.001/{year}/{month:02}/FLDAS_{model}_A_{aoi}_D.A{year}{month:02}{day:02}.001.nc'
    return path
def test_brp():
    spec = resource_spec
    print(build_resource_path(**spec))
test_brp()

In [ ]:
        
def get_url(spec, base_url=FLDAS_BASE_URL):
    resource_path = build_resource_path(**spec)
    return os.path.join(base_url, resource_path)
def test_get_url():
    spec = resource_spec
    url = get_url(spec)
    print(get_url(spec))

test_get_url()


In [ ]:
%%time
# url = ('https://hydro1.gesdisc.eosdis.nasa.gov/opendap/hyrax/FLDAS/'
#        'FLDAS_NOAH01_A_EA_D.001/2019/04/FLDAS_NOAH01_A_EA_D.A20190401.001.nc')
session = setup_session(username, pw, check_url=url)

### Use xarray.open_dataset with pydap.cas.urs's session object
- http://xarray.pydata.org/en/stable/io.html#opendap
- PydapDataStore [src](https://is.gd/Wkz1ob)

In [ ]:
%%time
store = xr.backends.PydapDataStore.open(url, session=session)
# ds = xr.open_dataset(store) # currently not working


In [ ]:
vars_info = store.get_variables()

In [ ]:
print(vars_info.keys()) #KeysView object
varname = 'SoilMoi00_10cm_tavg'
var = store.get(varname) # xr.Variable
print(var)

In [ ]:
# Recommneded to work with xr.DataArray or xr.DataSet
var = xr.DataArray(var)
print(var)

# HALT!

---

---

## Back to using the local files....
Modified: May 27, 2019

In [ ]:
## Read local netcdf files
## This works smoothly....
for m in np.arange(1,13)[:1]:
    fpath = Path(f'/home/hayley/data/FLDAS_NOAH01_A_EA_D.A201904{day:02}.001.nc')
    day_ds = xr.open_dataset(fpath)


In [ ]:
data = day_ds.load()
nodata = data.attrs.get('missing_value', None)
crs = data.attrs.get('MAP_PROJECTION',None)

print(nodata)
print(crs) # epsg 4082

In [ ]:
# Let's go holoviews
hv_dataset = hv.Dataset(data)
hv_dataset.to(hv.Image, kdims=['time','Y','X'])


In [ ]:
# Southern Africa Dataset
ds_sa = xr.open_mfdataset('/home/hayley/data/mint/FLDAS_NOAH01_A_SA_D.001/FLDAS_NOAH01_A_SA_D.*.nc4')
ds_sa

In [ ]:
# Eastern Africa Dataset
ds_ea = xr.open_mfdataset('/home/hayley/data/mint/FLDAS_NOAH01_A_EA_D.001/2017/08/*.nc')
print(ds_ea)
varnames = list(ds.variables.keys())
nprint('variables: ', varnames)

In [ ]:
ds1 = ds.sel(bnds=0)
ds2 = ds.sel(bnds=1)

In [ ]:
%%time
august_data = ds.drop_dims('bnds').load()
august_data


In [ ]:
soil = ds.SoilMoi00_10cm_tavg
p = soil.plot()

In [ ]:
hv_soil_ds = hv.Dataset(soil) # this will result in right mapping!
nprint('soil', soil.dims, soil)
nprint('hv_soil_ds', hv_soil_ds)
nprint('hv_soil_ds dims',hv_soil_ds.dimensions())

In [ ]:
# hv_soil_ds = hv.Dataset(soil, kdims=['time', 'Y', 'X']) # this will result in wrong mapping!
# nprint('hv_soil_ds dims',hv_soil_ds.dimensions())

In [ ]:
# hv_soil_ds = hv.Dataset(soil, kdims=['X', 'Y', 'time']) # this will result in wrong mapping!
# nprint('hv_soil_ds dims',hv_soil_ds.dimensions())

In [ ]:
f,ax= plt.subplots()

pt_lon, pt_lat = (20, 50)
soil_pt = soil.sel(X=pt_lon, Y=pt_lat, method='nearest')
soil_pt.plot(ax=ax, color='g')

In [ ]:
soil_yr_mean = soil.mean(dim='time')

In [ ]:
(soil_pt - soil_yr_mean.sel(X=pt_lon, Y=pt_lat, method='nearest')).plot(ax=ax, color='r')

In [ ]:
soil - soil_yr_mean

---
### Let's go holoviews
Using full 31 day's (august, 2019) dataset


In [ ]:
%opts Image [tools=['hover']]

In [ ]:
hv_dataset = hv.Dataset(ds)
print(hv_dataset)

In [ ]:
not_vars = ['X','Y','time', 'time_bnds']
for ele in not_vars:
    try:
        varnames.remove(ele)
    except ValueError as e:
        print(e)
print(varnames)
    


In [ ]:
# Create a table of variable names

# hv.Table({'varname': varnames})
t1 = hv.Table({'x': [1,2,3], 'y':[10,20,30]})

df = pd.DataFrame({'x': [1,2,3], 'y':[10,20,30]})
t2 = hv.Table(df)

t3 = hv.Table( ([1,2,3], [10,20,30]), 'col1', 'col2' )

t4 = hv.Table( [ (1,10), (2,20), (3,30), (4,40) ], 'col1', 'col2')

(t1 + t2 + t3 + t4).cols(3)

In [ ]:
t1 + t2 + t3 + t4

In [ ]:
def get_imgmap(varname):
    print(varname)
    varmap = gv.Dataset(ds[varname]).to(gv.Image, kdims=['X','Y']) #dynamic=True to make it dynamicmap
    varmap.label = varname
    return varmap

In [ ]:
map0 = get_imgmap(varnames[0])
print(map0.name)
print(map0.label)

In [ ]:
map1 = get_imgmap(varnames[1])
print(map1.name, varmap1.label)

In [ ]:
%%opts Image [width=500, height=500]
layout = (map0 + map1)

In [ ]:
layout

## holomaps for august fldas data
Modified: May 30, 2019

In [ ]:
august_fldas = august_data.copy()

In [ ]:
print(august_fldas)

In [ ]:
varname = 'SoilTemp00_10cm_tavg'

In [ ]:
august_fldas[varname]

In [ ]:
soil_temp_var = august_fldas['SoilTemp00_10cm_tavg']
hv_dataset = hv.Dataset(soil_temp_var,kdims = ['dim0', 'dim1', 'time'

In [ ]:
august_fldas

In [ ]:
def ds_pt(ds, long, lat):
    ds_pt = ds.sel(X=long, Y=lat, method='nearest')
    return ds_pt
    

august_longlat = ds_pt(august_data, pt_lon, pt_lat)

In [ ]:
def get_pt_curve(ds_pt, varname):
    hv_dataset = hv.Dataset(ds_pt[varname]) #kdim = time, vdim = varname
    return hv_dataset.to(hv.Curve,'time', varname)

gen_curve0 = lambda pt_lon, pt_lat: get_pt_curve(ds_pt(august_data, pt_lon, pt_lat), varnames[0])
gen_curve0(20,10)

In [ ]:
gen_curve1 = lambda pt_lon, pt_lat: get_pt_curve(ds_pt(august_data, pt_lon, pt_lat), varnames[1])
gen_curve1(20,10)

In [ ]:
# create variablename table
len(varnames)

In [ ]:
hv.extension('bokeh')

In [ ]:
list(ds.variables.keys())


In [ ]:
data.

---
## Xarray Workbook 1
Modified: May 28, 2019

This is my sketchpad to understand xr.DataArray and xr.Dataset constructors.

Keys:
    - `xarray`'s main motivation is to model after `netcdf` format
    - it handles pandas's limitation on 2D (or 3D at max) dataframe
    - it maintains the `pandas`'s named dimensions idea
    - we can think of it as a multidimensional array(`numpy.ndarray`) with named dimensions
    - each dimension has a `name` and `tick-marks`. These tick-marks are called `coordinates`. `numpy` doesn't have this feature, so all of its indexing is by integer/order based. In `xarray`, since we have a name for each dimension (ie. axis) as well as a list of coordinates (ie. tick-marks) for each dimension (again, ie. axis), we can refer to a value in the xrray DataArray container with more semantic-aware indexing
   

![](../assets/xarray_overview_.png)

In [ ]:
arr = np.array(np.matrix('0, 10; 1, 11; 2, 12'))
nprint(arr.shape, arr)
# np.c_[df, ['a','b','c']] #have you tried this?

## Simplest DataArray Constructor

In [ ]:
xarr = xr.DataArray(arr)
print(xarr)

# 1. more meaning ful dimension name
renamed = xarr.rename(dim_0='date', dim_1='station_id')
nprint('renamed', renamed)

Note `xarray` and `numpy` follow the same dimention assignment order. Dim0 is along the rows, Dim1 is along the columns

In [ ]:
renamed.coords

In [ ]:
# Let's assign some coordinates
coords = {
    'date': ['2019-01-01', '2019-01-02', '2019-01-03'],
    'station_id': ['LA', 'SF']
    
}

In [ ]:
new_xarr = renamed.assign_coords(**coords)
print(new_xarr)

In [ ]:
# I can store any relevant metadata as 'attrs'
new_xarr.attrs

In [ ]:
new_xarr.attrs.update(unit='km')
new_xarr.attrs.update(description='US State Daily Temperature Flux')
new_xarr.attrs.update(collector='NASA')
new_xarr.attrs.update(last_updated='2019-05-28')
new_xarr.attrs.update(license='MIT')


pprint(new_xarr.attrs)

## Let's specify these parameters at the construction time

In [ ]:
meta = new_xarr.attrs.copy()
nprint('meta data', meta)

In [ ]:
xarr2 = xr.DataArray(arr, 
                     dims = ['time', 'state'],
                     # coords as a list of tuples: each tuple = (dimname, coord_values)
                     # this results in setting coordinate name same as its dimension's name
                     # To set coord's name specifically, use a dictionary format
                     #   eg: coords = {coord_name1: coord_vals1, coord_name2, coord_vals2}
                     #   In this case, dimensions must be provided explicitly
                     #   See example below
                     coords=[('time', pd.date_range('2019-01-01', periods=3)),
                             ('state', ['LA', 'SF'])],
                     attrs=meta,
                     name='US state example data array'
                    )
print(xarr2)

## `Dimensions` and their `coordinates` 
DataArray Constructor
```
darr = xr.DataArray(
        data,
        dims=['dimname0', 'dimname1'],
        coords=
        attrs=
        name=
)
```

### Coordinates
1. A dictionary of form {'coordname': coord1, 'coordname2': coords2, ...}  
    - This requires the `dims` to be explicitly provided

In [ ]:
coords = {'coord1': pd.date_range('2019-05-05', periods=3),
          'coord2': ['LA','SF']
         }
darr = xr.DataArray(arr, 
             dims=['time', 'state'], # this will error because they `dims` must be a subset of `coords.keys()`
                    # when `coords` is given as a dictionary. Now I see why `coords` keyword is specified before 
                    # `dims`
             coords=coords,
            )
print(darr)

In [ ]:
# Let's fix the dimension names so that it works.
darr = xr.DataArray(arr,
                    coords=coords,
                    dims=['coord1', 'coord2'])
print(darr)

The advantage of using dictionary format for the `coords` is that we can specify extra coordinates that are about the dimensions (ie. axes).


In [ ]:
darr = xr.DataArray(arr, 
             dims=['time', 'state'],
             coords={
                 'time': pd.date_range('2019-05-05', periods=3),
                 'state': ['LA', 'SF'],
                 'const': 17 # more on this extra (dimension-independent) coordinate later
             }
            )
print(darr)

But those dimension-independent coordinates have constraints: 
- a coordinate must have a value of a non-iterable datatype (eg. 15, 0.01, etc but not [1,2,3]). 
    - It can have a name not in `dims`
- If a coordinate's value is an iterable, it's 
- If a coordinate can have a name that is not in `dims`, but its value must be a tuple (or other iterable) following the tuple constructor format for a coordinate that has a cooresponding dimension. 
    - Eg: coord3 = ('extra_coordname', ('dimname0', [1,2,3]))

Their usecases will be explained in more details later.

In [ ]:
# This is okay
darr = xr.DataArray(arr, 
             dims=['time', 'state'],
             coords={
                 'time': pd.date_range('2019-05-05', periods=3),
                 'state': ['LA', 'SF'],
                 'coord3': 'hihi' # try any other non-iterable datatypes: 0.01,'a', 'hihi'
             }
            )
print(darr)

In [ ]:
# This is not okay
darr = xr.DataArray(arr, 
             dims=['time', 'state'],
             coords={
                 'time': pd.date_range('2019-05-05', periods=3),
                 'state': ['LA', 'SF'],
                 'coord3': [0.01,1] # doesn't work because the value is an iterable 
             }
            )
print(darr)

In [ ]:
# This is okay
darr = xr.DataArray(arr, 
             dims=['time', 'state'],
             coords={
                 'time': pd.date_range('2019-05-05', periods=3),
                 'state': ['LA', 'SF'],
                 'coord3': ('time', [1,2,3]) # this works because 'time' is one of the dimensions
                 #but fails if the length of the iterable doesn't match 'time's length, eg. [1,2,3,4]. Try it.
             }
            )
print(darr)

In [ ]:
# This is okay
darr = xr.DataArray(arr, 
             dims=['time', 'state'],
             coords={
                 'time': pd.date_range('2019-05-05', periods=3),
                 'state': ['LA', 'SF'],
                 'coord3': ( ('time', 'state'), np.random.randn(6).reshape(3,2)) 
                 # okay because 'time' and 'state' are dimension names
             }
            )
print(darr)

### `xr.DataArray` constructor from `pd.DataFrame`
Precedant of propagating DataArray properties at construction time
    - args to the `xr.DataArray` constructor
    - non-specified arguments will be filled in from the `pandas` object

In [ ]:
df = pd.DataFrame(arr, columns=['LA', 'SF'], index=pd.date_range('2020-01-01', periods=3))
df

In [ ]:
darr = xr.DataArray(df)

In [ ]:
darr

Notice that `df`'s index is set to the first dimension (which is named `dim_0` by default)'s coordinate, and `df`'s column names to the coordinate of the second dimension (`dim_1`).

Let's try providing dimension names.

In [ ]:
darr = xr.DataArray(df, dims=['time', 'state'])
print(darr)

What if the input `pd.DataFrame` instance has default index and column names?


In [ ]:
df = pd.DataFrame(arr)
print(df)

In [ ]:
darr = xr.DataArray(df)
print(darr)

Same rule applies. That is, we use the input `df`'s index and columnnames to fill in non-specified filed for the new xr.DataArray object.

Let's see if specifying the coordinates correctly take a precedance over the input `df`'s index and columns.

In [ ]:
darr = xr.DataArray(df,
                    coords=[('time', pd.date_range('2021-01-01',periods=3)),
                            ('state', ['LA', 'SF'])],
                    #dims=['time', 'state'] # optional, as it's redundant
                   )
print(darr)

                            

Notice that the coordinates are set from the direct input arguments to `xr.DataArray` constructor.

### `xr.DataArray.rename` method
- returns a **new** xr.DataArray with the same (**NOT** a copied version of) data and modification on the properties

In [ ]:
print(darr)

In [ ]:
new_darr = darr.rename(state='us_state')
print(new_darr)

In [ ]:
print(darr is new_darr)

In [ ]:
# see that the underlying data is copied as well
print(darr.values is new_darr.values) # same as print(id(darr.values) == id(new_darr.values))

Suprise?! Is this really true?

In [ ]:
print(id(darr.values) == id(new_darr.values))

In [ ]:
# Let's see if changes in one array is reflected on the `renamed` DataArray's underlying data

In [ ]:
print(darr.values)

In [ ]:
darr.values[0,0] = -100
nprint(darr.values)
nprint(new_darr.values)

Okay. This is worthwhile to remember. 

> xr.DataArray.rename() will return a **new** instance with properties changed (eg. dimension names, coordinate values),
but the new instance will have to the **same** handle to the original dataarray's `values`(ie. the underlying data)!!

This means, changing the underlying data in one instance is directly reflected on the other instance. Nice in that the data is copied, but if we truely want a new instance, we need to figure out what's the right way to deep copy the underlying data.

## xarray's Dataset class
Keys:
- a variable is either `data_variable` (`data_var`) or `coordinate_variable`(`coords`)

    For example, in the diagram below, `temperature` and `precipitation` are `data variable` and all other arrays are `coordinate variables`
    <img src="../assets/xr_dataset_structure.png" alt="xr_dataset" width="500"/>


- multi-dimensional equivalent of a pd.Datafrmae + labelled axes
- a dict-link container of labelled arrays (ie.xr.DataArray objects) with *aligned* dimensions
- designed as an in-memory representation of the data model from the netCDF file format

4 main properties of a Dataset object
- dims: a dictionary mapping from dimension names to the fixed length of each dimension, eg: `{'dim0': 4, 'dim1': 3}`
- data_vars: a dict-link container of DataArrays corresponding to variables
- coords: a dict-link contain of DataArrays intended to label points used in `data_vars`
- attrs: OrderedDict to hold arbitrary metadata

- (xarray) `data_var` : `coord` = 'vdims' : 'kdims' (holoviews)
- How to decide whether a variable belongs to `data_vars` or `coords`"
    - coordinates indicate constant/fixed/independed quantities 
    - varying/measured/dependednt quantities belongs to data
- recall `coords.keys()` is a superset of `dims`

<img src="../assets/create_dataset.png" alt="create_dataset" width="650"/>

In [ ]:
temp = 70+10*np.random.randn(2,3,4)
precip = 5+2*np.random.randn(2,3,4)
lon = [[-99.81, -99.44, -99.23], 
       [-99.79, -99.34, -99.12]]
lat = [[42.24, 42.21, 42.19],
       [42.63, 42.59, 42.44]]

```python
data_vars = {
    'temperature': xr.DataArray(data=temp,
                                coords = [('x', [x_tick1, x_tick2]),
                                          ('y', [y_tick1, y_tick2, y_tick3]),
                                          ('t', [t_tick1, t_tick3, t_tick3, t_tick4])],
                                dims = ['x','y','time'], # optional as it's redundant
                                attrs = temp_metadata,
                                name = temp_name
                               ),
    'precipitation': xr.DataArray(data = precip,
                                  coords = [('x', [x_tick1, x_tick2]),
                                          ('y', [y_tick1, y_tick2, y_tick3]),
                                          ('time', [t_tick1, t_tick3, t_tick3, t_tick4])],
                                  dims = ['x','y','time'], # optional as it's redundant
                                 ),
}
```

Note that we don't really know what to use to tickmark values for `x` and `y` coordinates. These are very general coordinates, to indicate the general two dimensional space, and no semantics attached. So, we use the second syntax (which only requires a list of dimension names ('x', 'y', 't') and the underlying data for the data_variable) to construct each data variable:

```python
    {"varname": (`dims`, `underlying_data`),
    "varname2": (`dims`, `underlying_data`)}
    
```


In [ ]:
data_vars = {
    'temperature': (['x','y','time'], temp),
    'percipitation': (['x','y','time'], precip),
}                             
coord_vars = {
    'lon': (['x','y'], lon),
    'lat': (['x','y'], lat),
    
    # this is the last case (for the general coordinate variables, ie. x,y,t in our case
#     'x': [val1, val2] <-- what to put in..? whatever is, not very meaningful
#     'y': [ycoord1, ycoord2] <-- what to put in...? so we don't explicitly express these two
    'time': pd.date_range('2019-05-28', periods=4),
    'reference_time': pd.Timestamp('2019-05-27')
}
    

In [ ]:
ds = xr.Dataset(
    data_vars=data_vars,
                
    #coords should've named coord_vars, in my opinion
    coords = coord_vars
)

In [ ]:
ds

Pretty interesting. This is well-connected to the ideas behind holoviews `kdims` and `vdims`.

---
# Create `holoviews.Dataset` from `xarray.Dataset`

In [ ]:
ds

# Resources
- Pangeo architecture: [slides](https://is.gd/t9Rtqn)
    - Bring computation to the data (big data)
    - Uses `xarray`  which is supported by `Dask` in the backend
- Great tutorial on how to use OPeNDAP server with GES DISC (NASA's open data portal)
    - [link](https://is.gd/V4RJMS)
- xarray: read opendap data
    - [doc](http://xarray.pydata.org/en/stable/io.html#opendap)
    - use xarray.open_dataset() for password-protected Opendap files: [link](https://github.com/pydata/xarray/issues/1068)
    <img src="./assets/xarray-opendap.jpg" alt="xarray-opendap" width="500"/>
- xarray general tutorials
    - [liasa](http://pure.iiasa.ac.at/id/eprint/14952/1/xarray-tutorial-egu2017-answers.pdf)
    
